In [ ]:

%matplotlib widget
import matplotlib.pyplot as plt
from photometry_preprocessing import * 
from photometry_smoothing import *
from photometry_correcting import * 
from photometry_eventanalysis import * 
from photometry_variationanalysis import *
from photometry_visualization import * 
from scalebars import *
import deeplabcut_returnanalysis as dlcra 
from matplotlib.ticker import FormatStrFormatter
import pandas as pd 
import warnings
import functools
import operator
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import matplotlib
from matplotlib.lines import Line2D
matplotlib.rcParams['font.size'] = 10
plt.style.use('seaborn-deep')

In [ ]:
#paths to data folders 
FL = ['/scratch/rhs112/FL/RTGR-173_2022-02-23', '/scratch/rhs112/FL/RTGR-173_2022-04-17', '/scratch/rhs112/FL/RTGR-174_2022-02-24', 
    '/scratch/rhs112/FL/RTGR-174_2022-04-18', '/scratch/rhs112/FL/RTGR-176_2022-03-12', '/scratch/rhs112/FL/RTGR-176_2022-04-20', '/scratch/rhs112/FL/RTGR-121_2022-01-02']
ECD = ['/scratch/rhs112/ECD/RTGR-104_2022-01-21', '/scratch/rhs112/ECD/RTGR-177_2022-02-28', '/scratch/rhs112/ECD/RTGR-177_2022-04-21',
    '/scratch/rhs112/ECD/RTGR-178_2022-04-22','/scratch/rhs112/ECD/RTGR-179_2022-03-02', '/scratch/rhs112/ECD/RTGR-179_2022-04-23']
ECDControl = ['/scratch/rhs112/ECDControl/RTGR-191_2022-03-14', '/scratch/rhs112/ECDControl/RTGR-191_2022-03-30', '/scratch/rhs112/ECDControl/RTGR-192_2022-03-15',
            '/scratch/rhs112/ECDControl/RTGR-192_2022-03-31', '/scratch/rhs112/ECDControl/RTGR-193_2022-03-16', '/scratch/rhs112/ECDControl/RTGR-193_2022-04-01',
            '/scratch/rhs112/ECDControl/RTGR-200_2022-06-14', '/scratch/rhs112/ECDControl/RTGR-200_2022-06-18']
ECD13A = ['/scratch/rhs112/ECD1.3a/RTGR-108_2022-01-19', '/scratch/rhs112/ECD1.3a/RTGR-110_2022-02-02', '/scratch/rhs112/ECD1.3a/RTGR-182_2022-03-05',
        '/scratch/rhs112/ECD1.3a/RTGR-182_2022-04-02', '/scratch/rhs112/ECD1.3a/RTGR-182_2022-04-24', '/scratch/rhs112/ECD1.3a/RTGR-182_2022-04-29',
        '/scratch/rhs112/ECD1.3a/RTGR-184_2022-03-08', '/scratch/rhs112/ECD1.3a/RTGR-184_2022-04-04', '/scratch/rhs112/ECD1.3a/RTGR-184_2022-04-26']
WTFL = ['/scratch/rhs112/WTFL/RTGR-208_2022-08-01', '/scratch/rhs112/WTFL/RTGR-210_2022-08-03', 
        '/scratch/rhs112/WTFL/RTGR-208_2022-09-06', '/scratch/rhs112/WTFL/RTGR-209_2022-09-07',
       '/scratch/rhs112/WTFL/RTGR-210_2022-09-08']
WTFL_unfasted = ['/scratch/rhs112/WTFL/RTGR-208_2022-09-12', '/scratch/rhs112/WTFL/RTGR-209_2022-09-13',
                '/scratch/rhs112/WTFL/RTGR-210_2022-09-14']
WTECD = ['/scratch/rhs112/WTECD/RTGR-211_2022-08-04', '/scratch/rhs112/WTECD/RTGR-212_2022-08-05',
        '/scratch/rhs112/WTECD/RTGR-211_2022-09-09', '/scratch/rhs112/WTECD/RTGR-212_2022-09-10']
WTECD_unfasted = ['/scratch/rhs112/RTGR-211_2022-09-15']
ECD14A = ['/scratch/rhs112/ECD1.4a/RTGR-213_2022-08-08']
ECD14B = ['/scratch/rhs112/ECD1.4b/RTGR-215_2022-08-10', '/scratch/rhs112/ECD1.4b/RTGR-216_2022-08-11']
ECD14C = ['/scratch/rhs112/ECD1.4c/RTGR-217_2022-08-12']
ECD13A_anesthesia = ['/scratch/rhs112/anesthesia/RTGR-110_2021-12-30-4h-ane']
FL_anesthesia = ['/scratch/rhs112/anesthesia/RTGR-130-A-4h-20220608', '/scratch/rhs112/anesthesia/RTGR-161-A-4h-20220608'] 
ECD_anesthesia = ['/scratch/rhs112/anesthesia/RTGR-162-A-4h-20220606', '/scratch/rhs112/anesthesia/RTGR-163-A-4h-20220607', '/scratch/rhs112/anesthesia/RTGR-164-A-4h-20220607']
anesthesia = ECD_anesthesia + FL_anesthesia + ECD13A_anesthesia






# **Variation Analysis**
Comparison of variation in 465 signal across different behavioral states, e.g. immobility

In [ ]:
FL_variations = {'preDLCsleep': [], 'DLCsleep': [], 'postDLCsleep': [], 'premeal': [], 'meal': [], 'postmeal': []}
ECD_variations = {'preDLCsleep': [], 'DLCsleep': [], 'postDLCsleep': [], 'premeal': [], 'meal': [], 'postmeal': []}
ECDControl_variations = {'preDLCsleep': [], 'DLCsleep': [], 'postDLCsleep': [], 'premeal': [], 'meal': [], 'postmeal': []}
ECD13A_variations = {'preDLCsleep': [], 'DLCsleep': [], 'postDLCsleep': [], 'premeal': [], 'meal': [], 'postmeal': []}
ECD14C_variations = {'preDLCsleep': [], 'DLCsleep': [], 'postDLCsleep': [], 'premeal': [], 'meal': [], 'postmeal': []}
offsettime = 5 
iso_decrease = [7668-offsettime]
iso_increase = [8892-offsettime]

for path in WTFL:
    print(path)
    time = load_data(f'{path}/time.npy')
    signal = load_data(f'{path}/signal.npy')
    manualbehaviors, DLCbehaviors = load_processed_behaviors(f'{path}/', manual=False)  
    FL_variations['preDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_onsets'], [-120, 0])
    FL_variations['DLCsleep'] += calculate_OnOffVariation(signal, time, DLCbehaviors['immobility_onsets'], DLCbehaviors['immobility_offsets'])
    FL_variations['postDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_offsets'], [0, 120])
    try:
        FL_variations['premeal'] += calculate_WindowVariation(signal, time, manualbehaviors['meal_onsets'], [-120, 0])
        FL_variations['meal'] += calculate_OnOffVariation(signal, time, manualbehaviors['meal_onsets'], manualbehaviors['meal_offsets'])
        FL_variations['postmeal'] += calculate_WindowVariation(signal, time, manualbehaviors['meal_offsets'], [0, 120])
    except:
        FL_variations['premeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_onsets'], [-120, 0])
        FL_variations['meal'] += calculate_OnOffVariation(signal, time, DLCbehaviors['meal_onsets'], DLCbehaviors['meal_offsets'])
        FL_variations['postmeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_offsets'], [0, 120])
dfFL = pd.DataFrame({k:pd.Series(v) for k,v in FL_variations.items()})

for path in WTECD:
    print(path)
    time = load_data(f'{path}/time.npy')
    signal = load_data(f'{path}/signal.npy')
    manualbehaviors, DLCbehaviors = load_processed_behaviors(f'{path}/', manual=False)
    ECD_variations['preDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_onsets'], [-120, 0])
    ECD_variations['DLCsleep'] += calculate_OnOffVariation(signal, time, DLCbehaviors['immobility_onsets'], DLCbehaviors['immobility_offsets'])
    ECD_variations['postDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_offsets'], [0, 120])
    try:
        ECD_variations['premeal'] += calculate_WindowVariation(signal, time, manualbehaviors['meal_onsets'], [-120, 0])
        ECD_variations['meal'] += calculate_OnOffVariation(signal, time, manualbehaviors['meal_onsets'], manualbehaviors['meal_offsets'])
        ECD_variations['postmeal'] += calculate_WindowVariation(signal, time, manualbehaviors['meal_offsets'], [0, 120])
    except:
        ECD_variations['premeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_onsets'], [-120, 0])
        ECD_variations['meal'] += calculate_OnOffVariation(signal, time, DLCbehaviors['meal_onsets'], DLCbehaviors['meal_offsets'])
        ECD_variations['postmeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_offsets'], [0, 120])
dfECD = pd.DataFrame({k:pd.Series(v) for k,v in ECD_variations.items()})
        
for path in ECD14B:
    print(path)
    time = load_data(f'{path}/time.npy')
    signal = load_data(f'{path}/signal.npy')
    _, DLCbehaviors = load_processed_behaviors(f'{path}/', manual=False)
    ECDControl_variations['preDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_onsets'], [-120, 0])
    ECDControl_variations['DLCsleep'] += calculate_OnOffVariation(signal, time, DLCbehaviors['immobility_onsets'], DLCbehaviors['immobility_offsets'])
    ECDControl_variations['postDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_offsets'], [0, 120])
    ECDControl_variations['premeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_onsets'], [-120, 0])
    ECDControl_variations['meal'] += calculate_OnOffVariation(signal, time, DLCbehaviors['meal_onsets'], DLCbehaviors['meal_offsets'])
    ECDControl_variations['postmeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_offsets'], [0, 120])
dfECDControl = pd.DataFrame({k:pd.Series(v) for k,v in ECDControl_variations.items()})

'''
for path in ECD14A:
    print(path)
    time = load_data(f'{path}/time.npy')
    signal = load_data(f'{path}/signal.npy')
    _, DLCbehaviors = load_processed_behaviors(f'{path}/', manual=False)
    ECD13A_variations['preDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_onsets'], [-120, 0])
    ECD13A_variations['DLCsleep'] += calculate_OnOffVariation(signal, time, DLCbehaviors['immobility_onsets'], DLCbehaviors['immobility_offsets'])
    ECD13A_variations['postDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_offsets'], [0, 120])
    ECD13A_variations['premeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_onsets'], [-120, 0])
    ECD13A_variations['meal'] += calculate_OnOffVariation(signal, time, DLCbehaviors['meal_onsets'], DLCbehaviors['meal_offsets'])
    ECD13A_variations['postmeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_offsets'], [0, 120])
dfECD13A = pd.DataFrame({k:pd.Series(v) for k,v in ECD13A_variations.items()})
'''
for path in ECD14C:
    print(path)
    time = load_data(f'{path}/time.npy')
    signal = load_data(f'{path}/signal.npy')
    _, DLCbehaviors = load_processed_behaviors(f'{path}/', manual=False)
    ECD14C_variations['preDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_onsets'], [-120, 0])
    ECD14C_variations['DLCsleep'] += calculate_OnOffVariation(signal, time, DLCbehaviors['immobility_onsets'], DLCbehaviors['immobility_offsets'])
    ECD14C_variations['postDLCsleep'] += calculate_WindowVariation(signal, time, DLCbehaviors['immobility_offsets'], [0, 120])
    ECD14C_variations['premeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_onsets'], [-120, 0])
    ECD14C_variations['meal'] += calculate_OnOffVariation(signal, time, DLCbehaviors['meal_onsets'], DLCbehaviors['meal_offsets'])
    ECD14C_variations['postmeal'] += calculate_WindowVariation(signal, time, DLCbehaviors['meal_offsets'], [0, 120])
dfECD14C = pd.DataFrame({k:pd.Series(v) for k,v in ECD14C_variations.items()})





#df = pd.concat([dfFL, dfECD, dfECDControl, dfECD13A], axis=1, keys = ['FL', 'ECD', 'ECDControl', 'ECD13A'])
#df.columns = df.columns.rename(['sensor', 'behavior'])
#df = df.unstack().reset_index().drop('level_2', axis=1).dropna().rename(columns={0:'variation'})
#df = df.unstack().reset_index()


In [ ]:
FL_desc = dfFL.describe().loc[['mean', 'count'], :]
FL_sem = pd.DataFrame(dfFL.sem(axis=0)).rename(columns={0: 'sem'}).T
FL_desc = FL_desc.append(FL_sem).T.reset_index().rename(columns={'index': 'behavior'})
FL_desc['sensor'] = 'WT FL'

ECD_desc = dfECD.describe().loc[['mean', 'count'], :]
ECD_sem = pd.DataFrame(dfECD.sem(axis=0)).rename(columns={0: 'sem'}).T
ECD_desc = ECD_desc.append(ECD_sem).T.reset_index().rename(columns={'index': 'behavior'})
ECD_desc['sensor'] = 'WT ECD'

ECDControl_desc = dfECDControl.describe().loc[['mean', 'count'], :]
ECDControl_sem = pd.DataFrame(dfECDControl.sem(axis=0)).rename(columns={0: 'sem'}).T
ECDControl_desc = ECDControl_desc.append(ECDControl_sem).T.reset_index().rename(columns={'index': 'behavior'})
ECDControl_desc['sensor'] = 'ECD 1.4b'

'''
ECD13A_desc = dfECD13A.describe().loc[['mean', 'count'], :]
ECD13A_sem = pd.DataFrame(dfECD13A.sem(axis=0)).rename(columns={0: 'sem'}).T
ECD13A_desc = ECD13A_desc.append(ECD13A_sem).T.reset_index().rename(columns={'index': 'behavior'})
ECD13A_desc['sensor'] = 'ECD 1.4a'
'''
ECD14C_desc = dfECD14C.describe().loc[['mean', 'count'], :]
ECD14C_sem = pd.DataFrame(dfECD14C.sem(axis=0)).rename(columns={0: 'sem'}).T
ECD14C_desc = ECD14C_desc.append(ECD14C_sem).T.reset_index().rename(columns={'index': 'behavior'})
ECD14C_desc['sensor'] = 'ECD 1.4c'



combined_df = pd.concat([FL_desc, ECDControl_desc, ECD_desc, ECD14C_desc])#ECD13A_desc, 
combined_data = combined_df.pivot(index='behavior', columns='sensor', values='mean')
combined_err = combined_df.pivot(index='behavior', columns='sensor', values='sem')

#fig, ax = plt.subplots(1, figsize=(8,5))
#combineddata.plot(kind='bar', yerr=combinederr, ax=ax)


In [ ]:
order = ['WT FL', 'WT ECD', 'ECD 1.4b', 'ECD 1.4c']
sleep_data = combined_data.loc[['preDLCsleep', 'DLCsleep', 'postDLCsleep']].reindex(columns=order)
sleep_err = combined_err.loc[['preDLCsleep', 'DLCsleep', 'postDLCsleep']].reindex(columns=order)
meal_data = combined_data.loc[['premeal', 'meal', 'postmeal']].reindex(columns=order)
meal_err = combined_err.loc[['premeal', 'meal', 'postmeal']].reindex(columns=order)

fig, ax1 = plt.subplots(1, figsize=(8,5)) #dpi=1200 for final save 
sleep_data.plot(kind='bar', yerr=sleep_err, ax=ax1, legend=False, rot=0,
               error_kw=dict(capsize=4, capthick=1), color=['white', 'lightgrey', 'darkgrey', 'gray', 'dimgray'], edgecolor='black')
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.yaxis.set_ticks_position('left')
ax1.xaxis.set_ticks_position('bottom')
ax1.set_xticklabels(['Pre-Immobility', 'Immobility', 'Post-Immobility'])
ax1.set_ylabel('Coefficient of Variation', fontsize=12)
ax1.set_xlabel(None)
hl_list = [handle_label for handle_label in ax1.get_legend_handles_labels()]
newlabel_list = []
for label in hl_list[1]:
    n_val = combined_df.loc[(combined_df['sensor'] == label) & (combined_df['behavior'] == 'DLCsleep')]['count']
    n_val = int(n_val)
    newlabel_list.append(f'{label}, n={n_val}')
hl_list[1] = newlabel_list
ax1.legend(handles=hl_list[0], labels=hl_list[1], loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, handlelength=1, handleheight=1, edgecolor=None)
fig.subplots_adjust(right=0.7)
plt.savefig('NewSensorMealVariations.pdf', format='pdf', dpi=1200)

fig, ax2 = plt.subplots(1, figsize=(10,5)) #dpi=1200 for final save
meal_data.plot(kind='bar', yerr=meal_err, ax=ax2, legend=False, rot=0,
               error_kw=dict(capsize=4, capthick=1), color=['white', 'lightgrey', 'darkgrey', 'gray', 'dimgray'], edgecolor='black')
ax2.spines.right.set_visible(False)
ax2.spines.top.set_visible(False)
ax2.yaxis.set_ticks_position('left')
ax2.xaxis.set_ticks_position('bottom')
ax2.set_ylabel('Coefficient of Variation', fontsize=12)
ax2.set_xticklabels(['Pre-Meal', 'Meal', 'Post-Meal'])
ax2.set_xlabel(None)
hl_list = [handle_label for handle_label in ax1.get_legend_handles_labels()]
newlabel_list = []
for label in hl_list[1]:
    n_val = combined_df.loc[(combined_df['sensor'] == label) & (combined_df['behavior'] == 'meal')]['count']
    n_val = int(n_val)
    newlabel_list.append(f'{label}, n={n_val}')
hl_list[1] = newlabel_list
ax2.legend(handles=hl_list[0], labels=hl_list[1], loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, handlelength=1, handleheight=1, edgecolor=None)
fig.subplots_adjust(right=0.7)




In [ ]:
#Calculate variation pre-, during, and post anesthesia in a slope plot
from matplotlib.lines import Line2D
fig, ax4 = plt.subplots(1, figsize=(8,5))
for i in range(len(FL_variations['anesthesia'])):
    ax4.scatter(1, FL_variations['preanesthesia'][i], color='seagreen', alpha=0.5)
    ax4.scatter(2, FL_variations['anesthesia'][i], color='seagreen', alpha=0.5)
    ax4.scatter(3, FL_variations['postanesthesia'][i], color='seagreen', alpha=0.5)
    temp1 = ax4.plot([1,2,3], [FL_variations['preanesthesia'][i], FL_variations['anesthesia'][i], FL_variations['postanesthesia'][i]], color='green')

for i in range(len(ECD_variations['anesthesia'])):
    ax4.scatter(1, ECD_variations['preanesthesia'][i], color='salmon', alpha=0.5)
    ax4.scatter(2, ECD_variations['anesthesia'][i], color='salmon', alpha=0.5)
    ax4.scatter(3, ECD_variations['postanesthesia'][i], color='salmon', alpha=0.5)
    temp2 = ax4.plot([1,2,3], [ECD_variations['preanesthesia'][i], ECD_variations['anesthesia'][i], ECD_variations['postanesthesia'][i]], color='tomato')

for i in range(len(ECD13A_variations['anesthesia'])):
    ax4.scatter(1, ECD13A_variations['preanesthesia'][i], color='paleturquoise', alpha=0.5)
    ax4.scatter(2, ECD13A_variations['anesthesia'][i], color='paleturquoise', alpha=0.5)
    ax4.scatter(3, ECD13A_variations['postanesthesia'][i], color='paleturquoise', alpha=0.5)
    temp3 = ax4.plot([1,2,3], [ECD13A_variations['preanesthesia'][i], ECD13A_variations['anesthesia'][i], ECD13A_variations['postanesthesia'][i]], color='deepskyblue')
ax4.set_xticks([1, 2, 3])
ax4.set_xticklabels(['pre', 'Decrease in Anesthesia', 'post'])
ax4.set_ylabel('Coefficient of Variation')
ax4.set_title('Changes in Variation During Anesthesia')
legend_elements = [Line2D([0], [0], color = 'green', lw=1, label='FL'),
                   Line2D([0], [0], color = 'tomato', lw=1, label='ECD'),
                   Line2D([0], [0], color = 'deepskyblue', lw=1, label='ECD1.3a')]
ax4.legend(handles=legend_elements)